In [ ]:
from pokerkit import *

# Set up the initial game state
state = NoLimitTexasHoldem.create_state(
    # Automations
    (
        Automation.ANTE_POSTING,
        Automation.BET_COLLECTION,
        Automation.BLIND_OR_STRADDLE_POSTING,
        Automation.HOLE_CARDS_SHOWING_OR_MUCKING,
        Automation.HAND_KILLING,
        Automation.CHIPS_PUSHING,
        Automation.CHIPS_PULLING,
    ),
    False,  # False for big blind ante
    0, # ante
    (200, 400), # Small and big blinds
    400, # big bet
    (10000, 10000, 10000), # Starting chips for each player
    3 # Set the number of players
)

# Global vars
players = [x for x in state.player_indices]
small_blind_idx = 0 # start with first player as the small blind
big_blind_idx = small_blind_idx  + 1 # big blind is to left of small blind
starting_player_idx = big_blind_idx + 1 # starting player is player to left of big blind
game_over = False # flag to indicate if the game is over

def kelly_call_decision(prob_win: float, pot_size: float, call_amount: float, stack: float) -> float:
    """
    Determine how much of your stack you should be willing to risk calling based on the Kelly criterion.
    
    Parameters:
    - prob_win (float): Probability of winning (0 < prob_win < 1).
    - pot_size (float): Current pot size before your call.
    - call_amount (float): The amount you must put in to call.
    - stack (float): Your current stack size.
    
    Returns:
    float: The recommended maximum call amount based on Kelly criterion.
           If negative or zero, it suggests not calling.
    """
    # Calculate odds b = pot_size / call_amount
    if call_amount <= 0:
        # If call_amount is zero (e.g., free to call), just return 0 since there's no cost.
        # In reality, if it's free, just call, but here we just return 0 as there's no cost to consider.
        return 0.0

    b = pot_size / call_amount
    # Kelly fraction
    f = (prob_win * (b + 1) - 1) / b

    # If f <= 0, it's not profitable by Kelly standards to call.
    # If f > 0, risk up to f * stack.
    if f <= 0:
        return 0.0
    else:
        # The call_amount required might be more or less than f * stack. 
        # The Kelly suggestion would be to not risk more than f * stack.
        return min(call_amount, f * stack)


def kelly_open_bet_decision(prob_win: float, stack: float) -> float:
    """
    Determine how much to open-bet using a simplified Kelly approach when no bet is placed.
    Assumes a scenario where if called, you effectively get 1:1 on your money.
    
    Parameters:
    - prob_win (float): Probability of winning (0 < prob_win < 1).
    - stack (float): Your current stack size.
    
    Returns:
    float: The recommended bet amount based on the Kelly criterion.
           If negative or zero, it suggests not betting.
    """
    # In the simplified scenario b=1, so:
    # f = 2 * prob_win - 1
    f = 2 * prob_win - 1

    if f <= 0:
        # Not profitable to bet by Kelly standards
        return 0.0
    else:
        # Bet fraction f of your stack
        return f * stack

# Updated player_action function
def player_action(player, stage):
    # Get the player's cards and board cards
    board_cards = get_board_cards()
    player_hole_cards = get_player_hole_cards(current_player)

    # Calculate the player's hand strength
    hand_strength = calculate_hand_strength(
        state.player_count,
        parse_range(player_hole_cards),
        Card.parse(board_cards),
        2,
        5,
        Deck.STANDARD,
        (StandardHighHand,),
        sample_count=1000,
    )

    """Prompt player for an action with an enhanced and styled text-based UI."""
    hand = tuple(state.get_down_cards(player))
    
    # Display the header with the round name emphasized
    print("\033[1m" + "=" * 60 + "\033[0m")
    print("\033[1m" + f"{stage.upper()} ROUND".center(60) + "\033[0m")
    print("\033[1m" + "=" * 60 + "\033[0m")
    
    # Display game state information
    print("\033[1mGame Info\033[0m")
    print(f"Pot: {state.total_pot_amount}")
    print(f"Your Stack: {tuple(state.stacks)[player]}")
    print(f"Your Hand: {hand}")
    print(f"Board Cards: {tuple(state.get_board_cards(0))}")
    print(f"\nYour Hand Strength: {hand_strength:.3f}")
    print("\n\033[1mBetting Info\033[0m")
    print(f"Check/Call Amount: {state.checking_or_calling_amount}")
    print(f"Minimum Bet/Raise Amount: {state.min_completion_betting_or_raising_to_amount}")
     # Suggest Pot Bet or Call
    if state.checking_or_calling_amount == 0:
        pot_bet = kelly_open_bet_decision(hand_strength, state.stacks[current_player])
        print(f"Suggested Pot Bet: {pot_bet}")
    else:
        pot_call = kelly_call_decision(
            hand_strength,
            state.total_pot_amount,
            state.checking_or_calling_amount,
            state.stacks[current_player],
        )
        print(f"Suggested Pot Call: {pot_call}")
    print("\033[1m" + "=" * 60 + "\033[0m")
    
    # Loop for user input until a valid action is taken
    while True:
        print("\033[1mActions: [check] [call] [bet] [raise] [fold]\033[0m")
        action = input("\033[1mChoose an action:\033[0m ").strip().lower()
        print("\n")
        
        if action in {"check", "call"}:
            state.check_or_call()
            print(f"\033[1mYou chose to {action}.\033[0m")
            break
        elif action in {"bet", "raise"}:
            try:
                amount = int(input(f"\033[1mEnter {action} amount (minimum {state.min_completion_betting_or_raising_to_amount}):\033[0m "))
                if amount < state.min_completion_betting_or_raising_to_amount:
                    print(f"\033[1mAmount must be at least {state.min_completion_betting_or_raising_to_amount}. Try again.\033[0m")
                    continue
                state.complete_bet_or_raise_to(amount)
                print(f"\033[1mYou chose to {action} {amount}.\033[0m")
                break
            except ValueError:
                print("\033[1mInvalid input. Please enter a valid number.\033[0m")
        elif action == "fold":
            state.fold()
            print(f"\033[1mYou chose to fold.\033[0m")
            break
        else:
            print("\033[1mInvalid action. Please choose a valid option.\033[0m")


def get_player_hand(player):
    board_cards_tuple = tuple(state.get_board_cards(0))
    player_cards_tuple = tuple(state.hole_cards[player])
    board_cards = ""
    player_cards = ""

    value_map = {11: 'J', 12: 'Q', 13: 'K', 14: 'A'}
    suit_map = {'h': 'h', 'd': 'd', 'c': 'c', 's': 's'}

    for card in board_cards_tuple:
        value_str = value_map.get(card.rank, str(card.rank))  # Get face card or numeric value
        suit_str = suit_map[card.suit]                        # Get suit character
        board_cards += value_str + suit_str
    for card in player_cards_tuple:
        value_str = value_map.get(card.rank, str(card.rank))  # Get face card or numeric value
        suit_str = suit_map[card.suit]                        # Get suit character
        player_cards += value_str + suit_str
    hand = StandardHighHand.from_game(player_cards, board_cards)
    return hand

def get_player_hole_cards(player):
    player_cards_tuple = tuple(state.hole_cards[player])
    player_cards = ""

    value_map = {11: 'J', 12: 'Q', 13: 'K', 14: 'A'}
    suit_map = {'h': 'h', 'd': 'd', 'c': 'c', 's': 's'}
    for card in player_cards_tuple:
        value_str = value_map.get(card.rank, str(card.rank))  # Get face card or numeric value
        suit_str = suit_map[card.suit]                        # Get suit character
        player_cards += value_str + suit_str
    return player_cards

def get_board_cards():
    board_cards_tuple = tuple(state.get_board_cards(0))
    board_cards = ""

    value_map = {11: 'J', 12: 'Q', 13: 'K', 14: 'A'}
    suit_map = {'h': 'h', 'd': 'd', 'c': 'c', 's': 's'}

    for card in board_cards_tuple:
        value_str = value_map.get(card.rank, str(card.rank))  # Get face card or numeric value
        suit_str = suit_map[card.suit]                        # Get suit character
        board_cards += value_str + suit_str
    return board_cards

def find_winner():
    # Evaluate each player's hand and determine the winner
    best_hand = None
    winning_player = None

    print("\nFinal hands:")
    print(f"Board Cards: {tuple(state.get_board_cards(0))}\n")

    for player in state.player_indices:
        # display player's hole cards
        print(f"Player {player}: {tuple(state.get_down_cards(player))}\n")

        # Get the player's hand rank
        hand = get_player_hand(player)
        print(f"Player {player} hand rank: {hand}")

        # Determine the highest-ranked hand
        if best_hand is None or hand > best_hand:
            best_hand = hand
            winning_player = player

    print(f"The winner is Player {winning_player} with a hand rank of {best_hand}.")

def check_game_over():
    # Check if all players but one have folded
    count = 0
    for status in state.statuses:
        if status is True:
            count += 1
    
    # If only 1 player remains, the game is over
    if count == 1:
        return True
    else:
        return False

if __name__ == '__main__':
    # Game Header
    print("=" * 60)
    print(f"{'TEXAS HOLD’EM POKER':^60}")
    print("=" * 60)

    # Deal the initial hole cards to all players
    print("\nDealing initial hole cards...\n")
    for player in range(state.player_count * 2):
        state.deal_hole()
    print("Hole cards dealt successfully.")
    print("=" * 60)

    # Game Rounds
    for stage in ["pre-flop", "flop", "turn", "river"]:
        # Check if the game has ended early
        if game_over:
            print("[DEBUG] Game over detected. Ending game early.")
            break

        # Start the round
        print(f"\n{stage.upper()} ROUND".center(60, "="))
        if stage == "flop":
            state.burn_card()
            state.deal_board(3)  # Deal three community cards for the flop
        elif stage in ["turn", "river"]:
            state.burn_card()
            state.deal_board(1)  # Deal one community card for turn and river
        print(f"Cards dealt for {stage}.")

        # Player Actions for the Current Round
        while state.checking_or_calling_amount is not None:
            current_player = players[state.actor_index]
            print(f"\nPlayer {current_player}'s Turn")

            # Execute Player Action
            player_action(current_player, stage)

            # Check if the game is over (all players except one have folded)
            if check_game_over():
                game_over = True
                break

    # Determine the Winner
    print("\nFINAL RESULTS".center(60, "="))
    winner = -1
    for player_idx in range(len(state.statuses)):
        if state.statuses[player_idx]:
            winner = player_idx

    # Get the Winning Hand
    winning_hand = get_player_hand(winner)

    # Announce the Winner
    print(f"The Winner is Player {winner}!".center(60))
    print(f"Winning Hand: {winning_hand}".center(60))
    print("=" * 60)